# 序章
# Introduction

### ライブラリ・コマンドの準備
### Preparing for libralies and commands

In [ ]:
# 必要ライブラリのインポート
# Import required libralies

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### 初期設定
### Initial setting

In [ ]:
# デフォルトフォントサイズの変更
# Change default font size
plt.rcParams['font.size'] = 14

# warning表示off
# Ignore warning signs
import warnings
warnings.simplefilter('ignore')

In [ ]:
# デバイスの割り当て（8章）
# Device allocation (Chapter 8)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# 共通関数の読み込み（9章）
# Load common function (Capter 9)

# 共通関数のダウンロード
# Download common function
!git clone https://github.com/makaishi2/pythonlibs.git

# 共通関数のロード
# Load common function
from pythonlibs.torch_lib1 import *

# 共通関数の存在チェック
# Existance check for common function
print(README)

### 学習データ読み込み
### Load training data

In [ ]:
# 学習データの取り込み（12章）
# Load training data (Chapter 12)

# 学主要イメージのダウンロード
# Dowmload training images
w = !wget https://github.com/makaishi2/pythonlibs/raw/main/images/dog_wolf.zip
print(w[-2])

# 学習用イメージの展開
# Unzip training images
w = !unzip dog_wolf.zip
print(w[-1])

# ディレクトリ名の設定
# Set directory name
data_dir = 'dog_wolf'

# 学習とテスト用ディレクトリ名の設定
# Set train and test directory name
import os
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

# 分類席クラス名設定
# Set class names
classes = ['dog', 'wolf']

# 解答結果のツリー表示
# Show tree of dog-wolf images
!tree dog_wolf

In [ ]:
# 学習用データの準備（8章、10章、12章）
# Prepare for training data (Chapter 8, 10, 12)

# transformの定義
# Definition of transform

# 検証データ用：正規化のみ実施
# Set for test data: nomalization only 
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

# 訓練データ用：正規化に追加で反転とRandomErasingを実施
# Set for trainig data: nomalization, reverse and RandomErasing
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5), 
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)   
])

# datasetの定義
# Definition of datasets
train_data = datasets.ImageFolder(train_dir, transform=train_transform)
train_data2 = datasets.ImageFolder(train_dir, transform=test_transform)
test_data = datasets.ImageFolder(test_dir, transform=test_transform)

# dataloaderの定義
# Definition of dataloader
batch_size = 5
#train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
train_loader = DataLoader(train_data, batch_size=5, shuffle=True)
train_loader2 = DataLoader(train_data2, batch_size=40, shuffle=False)
test_loader = DataLoader(test_data, batch_size=5, shuffle=False)
test_loader2 = DataLoader(test_data, batch_size=10, shuffle=True)


### 学習データのイメージ表示
### Show the training images

In [ ]:
# 学習用データ（40件）
# Training data (40 images)
show_images_labels(train_loader2, classes, None, None)

### 検証データのイメージ表示
### Show the test images

In [ ]:
# 検証用データ（10件）
# Test data (10 images)
# 最初に正解なしでイメージのみ表示します
# First, show only the images without the correct answer
# それぞれの写真がイヌかオオカミか、各自判断してみて下さい
# Judge for yourself whether each image is of a dog or a wolf
torch_seed()
show_images_labels(test_loader2, ['', ''], None, None)

In [ ]:
# 検証用データ（10件）
# Test data (10 images)
# こちらが正解データです
# This shows correct answer
# 10件中何件正解できたでしょうか？
# How many of the 10 images did you answer correctly?
torch_seed()
show_images_labels(test_loader2, classes, None, None)

### モデル構築
### Model construction

In [ ]:
# 事前学習済みモデルを利用したモデル構築（11章）
# Model construction using pre-trained model (Chapter 10)

# 事前学習済みモデルの読み込み
# load pre-trained model
from torchvision import models
net = models.vgg19_bn(pretrained = True)

# 転移学習のためrequired_gradフラグをFalseにする
# Set required_grad flag to False for transfer learning
for param in net.parameters():
    param.requires_grad = False

# 乱数初期化
# Initialize ramdom numbers
torch_seed()

# 最終ノードの出力を2に変更する
# Set the output of final node to 2
in_features = net.classifier[6].in_features
net.classifier[6] = nn.Linear(in_features, 2)

# AdaptiveAvgPool2d関数の取り外し
# Remove AdaptiveAvgPool2d function
net.avgpool = nn.Identity()

# GPUの利用
# Set use of GPU
net = net.to(device)

# 学習率
# Training rate
lr = 0.001

# 損失関数に交差エントロピーを利用
# Use cross entropy for loss function
criterion = nn.CrossEntropyLoss()

# 最適化関数の定義
# Definition of optimization function

# パラメータ修正の対象を最終ノードに限定
# Limit parameter modification to final node
optimizer = optim.SGD(net.classifier[6].parameters(),lr=lr,momentum=0.9)

# historyファイルも同時に初期化する
# Initialize history file
history = np.zeros((0, 5))

In [ ]:
# 学習の実行（9章）
# Run training (Chapter 9)

# 繰り返し回数
# Set the number of repetitions
num_epochs = 10

# 学習（共通関数）
# training (common function)
history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history)

### 結果確認
### Result confirmation

In [ ]:
# 検証データを対象とした結果確認（9章 共通関数）
# Confirmation of results for test data (Chapter 9 common function)
torch_seed()
show_images_labels(test_loader2, classes, net, device)

### (参考) 共通関数の中身
### (Ref)　Detail of common function

In [ ]:
# 学習用関数
# Function for training
def fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history):

    # tqdmライブラリのインポート
    # Import tqdm library
    from tqdm.notebook import tqdm
    
    base_epochs = len(history)
  
    for epoch in range(base_epochs, num_epochs+base_epochs):
        # 1エポックあたりの正解数(精度計算用)
        # Number of correct answers per epoch (for precision calculation)
        n_train_acc, n_val_acc = 0, 0
        # 1エポックあたりの累積損失(平均化前)
        # Cumulative loss per epoch (before averaging)
        train_loss, val_loss = 0, 0
        # 1エポックあたりのデータ累積件数
        # Cumulative number of data per epoch 
        n_train, n_test = 0, 0

        # 訓練フェーズ
        # Start training
        net.train()

        for inputs, labels in tqdm(train_loader):
            # 1バッチあたりのデータ件数
            # Number of data per batch
            train_batch_size = len(labels)
            # 1エポックあたりのデータ累積件数
            # Comulative number of data per epoch
            n_train += train_batch_size
    
            # GPUヘ転送
            # Transfer to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 勾配の初期化
            # Initialize gradient
            optimizer.zero_grad()

            # 予測計算
            # Prediction calculation
            outputs = net(inputs)

            # 損失計算
            # Loss calculation
            loss = criterion(outputs, labels)

            # 勾配計算
            # Gradient calculation
            loss.backward()

            # パラメータ修正
            # Modify the parameter
            optimizer.step()

            # 予測ラベル導出
            # Predicted value calculation
            predicted = torch.max(outputs, 1)[1]

            # 平均前の損失と正解数の計算
            # Calculating loss before average and number of correct answers 
            # lossは平均計算が行われているので平均前の損失に戻して加算
            # Since the loss is averaged, add it back to the loss before the average. 
            train_loss += loss.item() * train_batch_size 
            n_train_acc += (predicted == labels).sum().item() 

        # 予測フェーズ
        # Start prediction
        net.eval()

        for inputs_test, labels_test in test_loader:
            # 1バッチあたりのデータ件数
            # Number of data per batch
            test_batch_size = len(labels_test)
            # 1エポックあたりのデータ累積件数
            # Comulative number of data per epoch 
            n_test += test_batch_size

            # GPUヘ転送
            # Transfer to GPU
            inputs_test = inputs_test.to(device)
            labels_test = labels_test.to(device)

            # 予測計算
            # Prediction calculation
            outputs_test = net(inputs_test)

            # 損失計算
            # Loss calculation
            loss_test = criterion(outputs_test, labels_test)
 
            # 予測ラベル導出
            # Predicted value calculation
            predicted_test = torch.max(outputs_test, 1)[1]

            #  平均前の損失と正解数の計算
            # lossは平均計算が行われているので平均前の損失に戻して加算
            val_loss +=  loss_test.item() * test_batch_size
            n_val_acc +=  (predicted_test == labels_test).sum().item()

        # 精度計算
        # Precision calculation
        train_acc = n_train_acc / n_train
        val_acc = n_val_acc / n_test
        # 損失計算
        # Loss calculation
        avg_train_loss = train_loss / n_train
        avg_val_loss = val_loss / n_test
        
        # 結果表示
        # Show results
        print (f'Epoch [{(epoch+1)}/{num_epochs+base_epochs}], loss: {avg_train_loss:.5f} acc: {avg_train_acc:.5f} val_loss: {avg_val_loss:.5f}, val_acc: {avg_val_acc:.5f}')
        
        # 記録
        # Recording
        item = np.array([epoch+1, avg_train_loss, avg_train_acc, avg_val_loss, avg_val_acc])
        history = np.vstack((history, item))
    return history


In [ ]:
# イメージとラベル表示
# Show images and labels
def show_images_labels(loader, classes, net, device):

    # DataLoaderから最初の1セットを取得する
    # Get first set from DataLoader 
    for images, labels in loader:
        break
    # 表示数は50個とバッチサイズのうち小さい方
    # The number of displayed images is 50 and the smaller of the batch size
    n_size = min(len(images), 50)

    if net is not None:
      # デバイスの割り当て
      # Set device
      inputs = images.to(device)
      labels = labels.to(device)

      # 予測計算
      # Prediction calculation
      outputs = net(inputs)
      predicted = torch.max(outputs,1)[1]
      #images = images.to('cpu')

    # 最初のn_size個の表示
    # Show the first n_size
    plt.figure(figsize=(20, 15))
    for i in range(n_size):
        ax = plt.subplot(5, 10, i + 1)
        label_name = classes[labels[i]]
        # netがNoneでない場合は、予測結果もタイトルに表示する
        # Show the result of prediction if net isn't None
        if net is not None:
          predicted_name = classes[predicted[i]]
          # 正解かどうか色分けする
          # Color-code whether the answer is cottect or not
          if label_name == predicted_name:
            c = 'k'
          else:
            c = 'b'
          ax.set_title(label_name + ':' + predicted_name, c=c, fontsize=20)
        # netがNoneの場合は、正解ラベルのみ表示
        # Show only corect labels if net is None
        else:
          ax.set_title(label_name, fontsize=20)
        # TensorをNumPyに変換
        # Transform Tensor to NumPy
        image_np = images[i].numpy().copy()
        # 軸の順番変更（channel, row, column) -> (row, column, channel）
        # Change axis order (channel, row, column) -> (row, column, channel)
        img = np.transpose(image_np, (1, 2, 0))
        # 軸の範囲を[-1, 1] -> [0, 1]に戻す
        # Revert the range of value to [-1, 1] -> [0, 1]
        img = (img + 1)/2
        #結果表示
        # Show results
        plt.imshow(img)
        ax.set_axis_off()
    plt.show()


In [ ]:
# Pytorch 乱数固定用
# For fixing random numbers of PyTorch

def torch_seed(seed=123):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.use_deterministic_algorithms = True
